In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
from scipy.signal import find_peaks, savgol_filter
from scipy.ndimage import binary_closing

In [3]:
import secrets      # cryptographically‑secure RNG
import base64       # for compact ASCII/“number + letter” output

def random_name(n_bytes: int = 16) -> str:
    # ① secure random bytes → ② Base32 text → ③ strip padding “=”
    token = base64.b32encode(secrets.token_bytes(n_bytes)).decode('ascii').rstrip('=')
    return f"{token}"

In [20]:
# Update your base directories
base_input_dir = r"D://Thesis//Mass_Line_Extraction//Outlier_Test//handwritten"
base_output_handwritten = r"D://Thesis//Mass_Line_Extraction//Outlier_Test_Result//Lines//handwritten"
base_output_printed = r"D://Thesis//Mass_Line_Extraction//Outlier_Test_Result//Lines//printed"
base_graph_folder = r"D://Thesis//Mass_Line_Extraction//Outlier_Test_Result//Graphs//handwritten"

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.signal import find_peaks, savgol_filter
from scipy.ndimage import binary_closing

def auto_savgol_smooth(profile, polyorder=2, spacing_factor=None,
                       plot=True, plot_title="", save_path=None,
                       show_thresholds=False, high_thresh=None, low_thresh=None):
    
    peaks, _ = find_peaks(profile, distance=8)
    if len(peaks) < 2:
        raise ValueError("Not enough peaks detected to estimate line spacing.")

    diffs = np.diff(peaks)
    eps = 1e-9
    weights = 1.0 / (diffs + eps)
    avg_spacing = int(np.round(np.sum(weights * diffs) / np.sum(weights)))

    # Dynamically estimate spacing_factor if not provided
    if spacing_factor is None:
        spacing_factor = min(max(1.2, avg_spacing / 20), 2.0)

    window_length = int(spacing_factor * avg_spacing)
    if window_length % 2 == 0:
        window_length += 1
    window_length = max(window_length, polyorder + 4)
    window_length = min(window_length,
                        len(profile) - 1 if len(profile) % 2 else len(profile) - 2)

    smoothed = savgol_filter(profile, window_length=window_length, polyorder=polyorder)

    if plot:
        fig = plt.figure(figsize=(14, 5))
        plt.plot(profile, label="Original", color="orange", alpha=0.6)
        plt.plot(smoothed, label=f"Smoothed (window={window_length})", color="blue")
        plt.plot(peaks, profile[peaks], "rx", label="Detected Peaks")

        if show_thresholds:
            if high_thresh is not None:
                plt.axhline(y=high_thresh, color="red", linestyle="--", label=f"High Thresh = {high_thresh:.2f}")
            if low_thresh is not None:
                plt.axhline(y=low_thresh, color="green", linestyle="--", label=f"Low Thresh = {low_thresh:.2f}")

        plt.title(plot_title or "Savitzky-Golay smoothing")
        plt.xlabel("Row Index")
        plt.ylabel("Sum of Pixel Intensities")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        if save_path is not None:
            fig.savefig(save_path)
        plt.close(fig)

    return smoothed, spacing_factor

def calculate_projection_profile_and_crop_lines_with_lines(image_path, folder_name):
    base_name = os.path.basename(image_path)
    subfolder_graph = os.path.join(base_graph_folder, folder_name)
    os.makedirs(subfolder_graph, exist_ok=True)
    output_path = os.path.join(subfolder_graph, f"{base_name}")
    image = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)

    if image is None:
        print(f"Error: Unable to load image {image_path}")
        return

    _, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    horizontal_projection = np.sum(binary_image, axis=1)

    # Smooth the projection profile (spacing_factor is inferred)
    smoothed, spacing_factor = auto_savgol_smooth(
        horizontal_projection,
        save_path=output_path,
        plot=True,
        show_thresholds=True
    )

    Q1 = np.percentile(smoothed, 25)
    Q3 = np.percentile(smoothed, 75)
    IQR = Q3 - Q1
    mean_val = np.mean(smoothed)
    min_val = np.min(smoothed)
    max_val = np.max(smoothed)

    # Individual estimates
    iqr_low = Q1 + 0.2 * IQR
    iqr_high = iqr_low + 0.35 * IQR

    mean_low = mean_val * 0.25
    mean_high = mean_val * 0.75

    scaled_low = min_val + 0.1 * (max_val - min_val)
    scaled_high = min_val + 0.45 * (max_val - min_val)

    low_thresh = np.median([iqr_low, mean_low, scaled_low])
    high_thresh = np.median([iqr_high, mean_high, scaled_high])

    # Re-run smoothing with thresholds for debug visualization
    smoothed, _ = auto_savgol_smooth(
        horizontal_projection,
        spacing_factor=spacing_factor,
        save_path=output_path,
        plot=True,
        show_thresholds=True,
        high_thresh=high_thresh,
        low_thresh=low_thresh
    )

    line_ranges = []
    is_in_line = False
    for row, value in enumerate(smoothed):
        if value > high_thresh and not is_in_line:
            start_row = row
            is_in_line = True
        elif value < low_thresh and is_in_line:
            end_row = row
            line_ranges.append((start_row, end_row))
            is_in_line = False
    if is_in_line:
        line_ranges.append((start_row, len(smoothed)))

    if line_ranges:
        line_ranges[0] = ((line_ranges[0][0]//2, line_ranges[0][0]))

    for i in range(1, len(line_ranges)):
        temp = (line_ranges[i - 1][1] + line_ranges[i][0]) // 2
        line_ranges[i - 1] = (line_ranges[i - 1][0], temp)
        line_ranges[i] = (temp, line_ranges[i][1])

    def ensure_and_save_crop(root_folder, cropped_line):
        subfolder = os.path.join(root_folder, folder_name)
        os.makedirs(subfolder, exist_ok=True)
        name = random_name()
        cv2.imwrite(os.path.join(subfolder, f"{name}.png"), cropped_line)

    for start, end in line_ranges:
        cropped_line = image[start:end, :]
        ensure_and_save_crop(base_output_handwritten, cropped_line)

In [22]:
def process_all_images():
    for root, dirs, files in os.walk(base_input_dir):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                image_path = os.path.join(root, file)
                folder_name = os.path.basename(root)
                try:
                    calculate_projection_profile_and_crop_lines_with_lines(image_path, folder_name)
                except Exception as e:
                    print(f"Error processing {image_path}: {e}")

In [24]:
process_all_images()

In [24]:
calculate_projection_profile_and_crop_lines_with_lines("D://Thesis//Mass_Line_Extraction//output_images//printed//022//a02-004.png", "D://Thesis//Mass_Line_Extraction//Single_temporary")